In [10]:
import kagglehub
import pandas as pd
import numpy as np
 # Download latest version
path = kagglehub.dataset_download("wenruliu/adult-income-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Usuario\.cache\kagglehub\datasets\wenruliu\adult-income-dataset\versions\2


In [11]:
df = pd.read_csv(fr'{path}\adult.csv')

def prepocessing(df):

    df.drop(columns=['fnlwgt', 'education', 'capital-gain', 'capital-loss', 'hours-per-week'], inplace = True)

    fill_rows = ~(df == '?').any(axis = 1)
    df = df[fill_rows]
    
    for col in df.columns:
        if df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col]):
          df[col], _ = pd.factorize(df[col])

    return df

def split_train_test(df, test_size=0.3):
    # 1. Embaralha o DataFrame inteiro aleatoriamente
    # Usa sample(frac=1) para selecionar 100% dos dados de forma aleatória
    # random_state é usado para reprodutibilidade (sempre a mesma ordem se o número for o mesmo)
    shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # 2. Calcula o ponto de corte
    len_df = len(shuffled_df)
    cut_point = len_df - round(len_df * test_size)
    
    # 3. Divide o DataFrame embaralhado
    train = shuffled_df[:cut_point]
    test = shuffled_df[cut_point:]
    
    print(f'Tamanho total: {len(df)}')
    print(f'Tamanho do treino: {len(train)}')
    print(f'Tamanho do teste: {len(test)}')
    
    return train, test

df = prepocessing(df)
X_train, X_test = split_train_test(df)

Tamanho total: 45222
Tamanho do treino: 31655
Tamanho do teste: 13567


C:\Users\Usuario\AppData\Local\Temp\ipykernel_16868\1569553445.py:11: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col]):
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16868\1569553445.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col], _ = pd.factorize(df[col])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16868\1569553445.py:11: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col]):
C:\Users\Usuari

In [ ]:
import numpy as np
import pandas as pd

class KNN_DP():
    def __init__(self, k):
        self.k = k
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        # Armazena os dados de treino
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def euclidian_dist(self, row1, row2):
        # Cálculo simples e robusto da distância euclidiana
        # Raiz da soma das diferenças ao quadrado
        diff = row1 - row2
        return np.sqrt(np.sum(diff**2))

    def get_vizinhos(self, test_row):
        distancias = []
        # Itera sobre todo o treino (pode ser lento, mas obedece "sem bibliotecas de ML")
        for i in range(len(self.X_train)):
            dist = self.euclidian_dist(test_row, self.X_train[i])
            distancias.append((self.y_train[i], dist))
        
        # Ordena pela distância (menor para maior)
        distancias.sort(key=lambda x: x[1])
        
        # Pega os k primeiros
        vizinhos = distancias[:self.k]
        return vizinhos

    def predict_instance(self, test_row):
        # KNN Tradicional
        vizinhos = self.get_vizinhos(test_row)
        classes_vizinhas = [v[0] for v in vizinhos]
        
        # Votação majoritária simples
        valores, contagens = np.unique(classes_vizinhas, return_counts=True)
        indice_vencedor = np.argmax(contagens)
        return valores[indice_vencedor]
    
    def predict_instance_private(self, test_row, epsilon):
        # KNN com Privacidade Diferencial (Mecanismo de Laplace na Votação)
        vizinhos = self.get_vizinhos(test_row)
        classes_vizinhas = [v[0] for v in vizinhos]
        
        # Obtém as classes possíveis e suas contagens reais
        valores_unicos = np.unique(self.y_train) # Todas as classes possíveis do problema
        
        # Dicionário de contagem inicializado com 0
        contagens = {val: 0 for val in valores_unicos}
        
        # Contagem real dos vizinhos
        for cls in classes_vizinhas:
            contagens[cls] += 1
            
        # Aplicação do Ruído de Laplace
        # Sensibilidade (Delta f) para votação é 1 (mudar 1 vizinho muda o voto em 1)
        # Scale (b) = Delta f / epsilon = 1 / epsilon
        scale = 1.0 / epsilon
        
        melhor_classe = None
        maior_voto_ruidoso = -float('inf')
        
        for cls in contagens:
            # Gera ruído de Laplace
            ruido = np.random.laplace(loc=0.0, scale=scale)
            voto_ruidoso = contagens[cls] + ruido
            
            if voto_ruidoso > maior_voto_ruidoso:
                maior_voto_ruidoso = voto_ruidoso
                melhor_classe = cls
                
        return melhor_classe

    def predict(self, X_test):
        # Predição padrão
        X_test = np.array(X_test)
        predicoes = []
        for row in X_test:
            pred = self.predict_instance(row)
            predicoes.append(pred)
        return np.array(predicoes)

    def predict_private(self, X_test, epsilon):
        # Predição com DP
        X_test = np.array(X_test)
        predicoes = []
        for row in X_test:
            pred = self.predict_instance_private(row, epsilon)
            predicoes.append(pred)
        return np.array(predicoes)

# ---------------------------------------------------------
# EXECUTANDO O ALGORITMO (Simulação do fluxo pedido)
# ---------------------------------------------------------

# Supondo que você já tenha X_train, X_test, y_train, y_test separados
# Ajuste as variáveis abaixo com seus dados reais
# K = 10 conforme a instrução

y_train= X_train['age'].values
x_treino = X_train.drop(columns = ['age']).values
y_test= X_train['age'].values
x_teste = X_train.drop(columns = ['age']).values
k = 10 
model = KNN_DP(k=k)

# Treinamento
print("Treinando modelo...")
model.fit(X_train, y_train)

# 1. Execução KNN Tradicional
print("Executando KNN Tradicional...")
y_pred_padrao = model.predict(X_test)

# Salvando resultado tradicional
df_resultado = pd.DataFrame({
    'y_true': y_test,
    'y_pred': y_pred_padrao
})
df_resultado.to_csv('resultado_knn_tradicional.csv', index=False)
print("Resultado Tradicional salvo.")

# 2. Execução KNN Privado para diferentes epsilons
epsilons = [0.5, 1, 5, 10]

for eps in epsilons:
    print(f"Executando KNN Privado (epsilon={eps})...")
    y_pred_privado = model.predict_private(X_test, epsilon=eps)
    
    # Salvando resultado privado
    nome_arquivo = f'resultado_knn_laplace_eps_{eps}.csv'
    df_resultado_priv = pd.DataFrame({
        'y_true': y_test,
        'y_pred': y_pred_privado
    })
    df_resultado_priv.to_csv(nome_arquivo, index=False)
    print(f"Arquivo {nome_arquivo} salvo.")

print("Processo finalizado.")

Treinando modelo...


NameError: name 'y_train' is not defined

In [ ]:
def acuracia(pred, y_target):
    return  sum(pred == y_target)/len(y_target)


,age,workclass,educational-num,marital-status,occupation,relationship,race,gender,native-country,income
31655,47,0,9,4,6,3,1,1,0,0
31656,40,4,13,1,4,4,0,1,0,0
31657,30,0,13,0,9,2,1,0,0,0
31658,19,0,10,0,3,5,1,1,0,0
31659,58,2,10,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
45217,39,2,6,3,5,2,0,0,0,0
45218,36,1,14,1,4,1,1,0,0,1
45219,53,1,5,1,7,1,1,0,0,0
45220,57,0,11,0,3,0,1,1,0,0
